In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [9]:
df = pd.read_csv('/kaggle/input/hack-chunk-desc/data/train_segments.csv')

In [10]:
df

,Advertisement ID,Segment_num
0,1831845,16
1,4467135,11
2,2285593,0
3,3825366,0
4,2313008,0
...,...,...
17735,4299761,1
17736,2298577,0
17737,4018035,1
17738,3922868,0


In [13]:
segments = pd.read_excel('/kaggle/input/hack-chunk-desc/data/segment_dict.xlsx')

In [14]:
import os

all_files = []
base_path = '/kaggle/input/hack-chunk-'

for i in ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10-stranger', '11', '12-stranger', '13', '14', '15', '16', '17'):
    folder_path = f"{base_path}{i}"
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            full_path = os.path.join(root, file)
            all_files.append(full_path)

In [15]:
id_to_path = {}
for file_path in all_files:
    file_id = int(file_path.split('/')[-1].split('.')[0])
    id_to_path[file_id] = file_path

def get_path(ad_id):
    return id_to_path.get(ad_id, None)

In [16]:
df['path'] = df['Advertisement ID'].apply(get_path)

In [17]:
df

,Advertisement ID,Segment_num,path
0,1831845,16,/kaggle/input/hack-chunk-2/2/1831845.mp4
1,4467135,11,/kaggle/input/hack-chunk-16/16/4467135.avi
2,2285593,0,/kaggle/input/hack-chunk-5/5/2285593.wav
3,3825366,0,/kaggle/input/hack-chunk-8/8/3825366.avi
4,2313008,0,/kaggle/input/hack-chunk-6/6/2313008.wav
...,...,...,...
17735,4299761,1,/kaggle/input/hack-chunk-16/16/4299761.avi
17736,2298577,0,/kaggle/input/hack-chunk-5/5/2298577.wav
17737,4018035,1,/kaggle/input/hack-chunk-10-stranger/10/401803...
17738,3922868,0,/kaggle/input/hack-chunk-9/9/3922868.avi


In [18]:
df = df[~df['path'].str.endswith('.jpg')]
df = df[~df['path'].str.endswith('.wav')]

In [19]:
df

,Advertisement ID,Segment_num,path
0,1831845,16,/kaggle/input/hack-chunk-2/2/1831845.mp4
1,4467135,11,/kaggle/input/hack-chunk-16/16/4467135.avi
3,3825366,0,/kaggle/input/hack-chunk-8/8/3825366.avi
5,3959329,0,/kaggle/input/hack-chunk-9/9/3959329.avi
6,4321218,1,/kaggle/input/hack-chunk-16/16/4321218.avi
...,...,...,...
17734,4227145,1,/kaggle/input/hack-chunk-14/14/4227145.avi
17735,4299761,1,/kaggle/input/hack-chunk-16/16/4299761.avi
17737,4018035,1,/kaggle/input/hack-chunk-10-stranger/10/401803...
17738,3922868,0,/kaggle/input/hack-chunk-9/9/3922868.avi


In [20]:
df['Segment_num'] = df['Segment_num'].replace(6, 0)

In [21]:
video_path = df.path[0]

In [22]:
! pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 45.6 MB/s eta 0:00:00:00:0100:01


In [23]:
import av
import torch
import numpy as np
from transformers import AutoProcessor, AutoModel
from huggingface_hub import hf_hub_download

np.random.seed(42)

2024-05-18 10:58:05.031903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 10:58:05.031998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 10:58:05.191213: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [24]:
def read_video_pyav(container, indices):
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]

    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)

    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)

    return indices

In [25]:
processor = AutoProcessor.from_pretrained("microsoft/xclip-base-patch32")
model = AutoModel.from_pretrained("microsoft/xclip-base-patch32")
model.to('cuda')

preprocessor_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/786M [00:00<?, ?B/s]

XCLIPModel(
  (text_model): XCLIPTextTransformer(
    (embeddings): XCLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): XCLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x XCLIPEncoderLayer(
          (self_attn): XCLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): XCLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps

In [26]:
import cv2

def get_video_frames(video_path, frame_indices):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file '{video_path}'")
        return None
    
    frames = []
    for i in range(max(frame_indices) + 1):
        ret, frame = cap.read()
        if not ret:
            break
        if i in frame_indices:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)

    cap.release()
    return frames

In [27]:
def getFeatures(video_path):    
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    
    cap.release()
    
    indices = sample_frame_indices(clip_len=8, frame_sample_rate=3, seg_len=total_frames)
    video = get_video_frames(video_path, indices)    

    inputs = processor(videos=list(video), return_tensors="pt").to('cuda')    
    video_features = model.get_video_features(**inputs).detach()
    
    return video_features

In [28]:
%%time

getFeatures(video_path)

/opt/conda/lib/python3.10/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  return torch.tensor(value)


CPU times: user 1.15 s, sys: 112 ms, total: 1.27 s
Wall time: 1.92 s


tensor([[ 8.1225e-01,  5.1156e-01, -7.8414e-01,  1.3853e-01,  1.0148e+00,
          1.5159e+00,  1.1154e+00, -1.7493e-01, -2.7552e-01,  3.3860e-01,
         -1.5913e+00, -8.5087e-01,  9.9745e-01,  1.0559e+00, -4.9369e-01,
         -7.5835e-01, -8.7456e-02, -2.5221e+00, -1.1565e+00, -1.4098e+00,
          1.1254e+00, -2.7539e-01, -8.1794e-01, -1.2172e+00, -5.8717e-01,
         -7.4409e-01,  1.3258e+00, -1.3408e+00, -7.3568e-01,  6.0635e-01,
          6.9922e-01, -1.8381e+00,  4.2565e-01,  9.6076e-01,  6.2486e-02,
         -1.0136e+00, -1.1235e+00, -1.8627e+00, -4.0384e-02,  6.6518e-01,
         -8.5970e-01,  1.3082e+00,  6.7061e-01,  1.8860e-01,  6.2070e-01,
         -2.5197e-01,  4.3151e-01, -8.0309e-02, -1.1289e+00, -1.3158e+00,
          9.1559e-01, -5.4867e-01, -3.8309e-01,  2.5348e-01,  6.1839e-01,
         -2.6026e-01,  5.8006e-01,  9.3805e-01,  1.3489e+00,  4.6038e-01,
          2.5918e-02, -1.9392e+00,  3.8811e-01,  8.2689e-02, -2.3351e-01,
         -1.4135e+00, -4.5634e-01, -1.

In [29]:
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split

X = df.drop('Segment_num', axis=1)
y = df['Segment_num']

X_train, X_test_val, y_train, y_train_val = train_test_split(X, y, test_size=0.05, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_test_val, y_train_val, test_size=0.5, random_state=42)

rus = RandomUnderSampler(random_state=42)
X_res, y_res = rus.fit_resample(X_train, y_train)

df_train = pd.concat([X_res, y_res], axis=1)
df_val = pd.concat([X_valid, y_valid], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

In [30]:
len(df_train)

1494

In [31]:
len(df_val)

355

In [32]:
len(df_test)

355

In [33]:
from tqdm import tqdm

def process_in_chunks(df, chunk_size, func):    
    num_chunks = int(np.ceil(len(df) / chunk_size))
    results = []    
    for i in tqdm(range(num_chunks)):        
        start = i * chunk_size
        end = min((i + 1) * chunk_size, len(df))        
        chunk = df.iloc[start:end].copy()
        try:            
            chunk['embs'] = chunk['path'].apply(func)
        except:            
            print(f'something goes wrong with {i}-th chunk')
        results.append(chunk)        
    return pd.concat(results)

In [34]:
chunk_size = 10

In [35]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [36]:
df_train_processed = process_in_chunks(df_train, chunk_size, getFeatures)

 22%|██▏       | 33/150 [02:21<08:06,  4.16s/it][mpeg4 @ 0x559557933c00] ac-tex damaged at 4 3
[mpeg4 @ 0x559557933c00] Error at MB: 40
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code
[mpeg4 @ 0x559557933c00] marker does not match f_code


something goes wrong with 33-th chunk


 34%|███▍      | 51/150 [03:32<05:14,  3.18s/it]

something goes wrong with 50-th chunk


 35%|███▌      | 53/150 [03:41<05:53,  3.65s/it]

...

 96%|█████████▌| 144/150 [12:53<00:35,  5.84s/it][NULL @ 0x559566f17c40] Invalid NAL unit size (9437 > 2947).
[NULL @ 0x559566f17c40] missing picture in access unit with size 2951
[h264 @ 0x5595673607c0] Invalid NAL unit size (9437 > 2947).
[h264 @ 0x5595673607c0] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55956722ae00] stream 1, offset 0xf0195a: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x55956722ae00] stream 1, offset 0xf01ac2: partial file
 97%|█████████▋| 145/150 [12:57<00:26,  5.36s/it]

something goes wrong with 144-th chunk


 97%|█████████▋| 146/150 [13:06<00:25,  6.31s/it][NULL @ 0x559567368400] Invalid NAL unit size (69578 > 40860).
[NULL @ 0x559567368400] missing picture in access unit with size 40864
[h264 @ 0x559566eacd00] Invalid NAL unit size (69578 > 40860).
[h264 @ 0x559566eacd00] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x5595671abf00] stream 0, offset 0xf0702e: partial file
 98%|█████████▊| 147/150 [13:15<00:21,  7.23s/it]

something goes wrong with 146-th chunk


100%|██████████| 150/150 [13:31<00:00,  5.41s/it]


In [37]:
df_val_processed = process_in_chunks(df_val, chunk_size, getFeatures)

100%|██████████| 36/36 [03:01<00:00,  5.05s/it]


In [38]:
df_train_processed = df_train_processed.dropna()
df_val_processed = df_val_processed.dropna()

In [39]:
from catboost import CatBoostClassifier

In [40]:
df_train_processed['embs_np'] = df_train_processed['embs'].apply(lambda x: x.cpu().flatten().numpy())

In [41]:
df_val_processed['embs_np'] = df_val_processed['embs'].apply(lambda x: x.cpu().flatten().numpy())

In [42]:
df_train_processed = df_train_processed.reset_index()
df_train_new = pd.concat([df_train_processed['embs_np'], df_train_processed['Segment_num']], axis=1)
array_cols = pd.DataFrame(df_train_new['embs_np'].to_list(), columns=[f'array_feature_{i}' for i in range(len(df_train_new['embs_np'][0]))])
df_train_new = pd.concat([df_train_new.drop(columns=['embs_np']), array_cols], axis=1)

X_train = df_train_new.drop(columns=['Segment_num'])
y_train = df_train_new['Segment_num']

In [43]:
df_val_processed = df_val_processed.reset_index()
df_val_new = pd.concat([df_val_processed['embs_np'], df_val_processed['Segment_num']], axis=1)
array_cols = pd.DataFrame(df_val_new['embs_np'].to_list(), columns=[f'array_feature_{i}' for i in range(len(df_val_new['embs_np'][0]))])
df_val_new = pd.concat([df_val_new.drop(columns=['embs_np']), array_cols], axis=1)

X_val = df_val_new.drop(columns=['Segment_num'])
y_val = df_val_new['Segment_num']

In [44]:
CatBoostModel_emb = CatBoostClassifier(
    iterations=1000,
    #learning_rate=0.03,
    depth=5,
    #early_stopping_rounds=200,
    use_best_model=True,
    loss_function='MultiClass',
    #eval_metric='F1',
    custom_metric='TotalF1:average=Macro',
    random_seed=42,
    task_type='GPU',
    #embedding_features=['embs_np']
)

In [45]:
CatBoostModel_emb.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    verbose=100
)

Learning rate set to 0.108743
0:	learn: 2.8496978	test: 2.8357910	best: 2.8357910 (0)	total: 224ms	remaining: 3m 43s
100:	learn: 1.1592948	test: 1.8980417	best: 1.8980417 (100)	total: 2.46s	remaining: 21.9s
200:	learn: 0.7001700	test: 1.7095435	best: 1.7086467 (199)	total: 4.35s	remaining: 17.3s
300:	learn: 0.4678453	test: 1.6075601	best: 1.6067632 (299)	total: 6.33s	remaining: 14.7s
400:	learn: 0.3380741	test: 1.5677960	best: 1.5676662 (399)	total: 8.18s	remaining: 12.2s
500:	learn: 0.2522279	test: 1.5271319	best: 1.5270530 (494)	total: 10s	remaining: 9.97s
600:	learn: 0.1914431	test: 1.4968360	best: 1.4968360 (600)	total: 11.9s	remaining: 7.88s
700:	learn: 0.1522854	test: 1.4688047	best: 1.4686984 (698)	total: 13.7s	remaining: 5.84s
800:	learn: 0.1236777	test: 1.4485094	best: 1.4483543 (799)	total: 15.5s	remaining: 3.86s
900:	learn: 0.1016410	test: 1.4313536	best: 1.4309999 (897)	total: 17.4s	remaining: 1.91s
999:	learn: 0.0855736	test: 1.4226417	best: 1.4198601 (971)	total: 19.2s	re

In [46]:
CatBoostModel_emb.get_best_score()

{'learn': {'TotalF1:average=Macro': 1.0, 'MultiClass': 0.08557355489416137},
 'validation': {'TotalF1:average=Macro': 0.37120479262747386,
  'MultiClass': 1.4198600661586707}}

In [65]:
CatBoostModel_emb.save_model('cb')

In [47]:
df_test_processed = process_in_chunks(df_test, chunk_size, getFeatures)

 22%|██▏       | 8/36 [00:44<02:32,  5.46s/it][NULL @ 0x559566defb00] Invalid NAL unit size (1378 > 729).
[NULL @ 0x559566defb00] missing picture in access unit with size 733
[h264 @ 0x559552df8ac0] Invalid NAL unit size (1378 > 729).
[h264 @ 0x559552df8ac0] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x559552fb8580] stream 0, offset 0xf00289: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x559552fb8580] stream 1, offset 0xf00cab: partial file
 25%|██▌       | 9/36 [00:48<02:19,  5.15s/it]

something goes wrong with 8-th chunk


 56%|█████▌    | 20/36 [01:44<01:22,  5.14s/it][NULL @ 0x5595677a1f80] Invalid NAL unit size (4987 > 753).
[NULL @ 0x5595677a1f80] missing picture in access unit with size 757
[h264 @ 0x559566df0d80] Invalid NAL unit size (4987 > 753).
[h264 @ 0x559566df0d80] Error splitting the input into NAL units.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x559552e66140] stream 1, offset 0xf0108a: partial file
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x559552e66140] stream 1, offset 0xf011e3: partial file
 58%|█████▊    | 21/36 [01:54<01:39,  6.61s/it]

something goes wrong with 20-th chunk


100%|██████████| 36/36 [03:07<00:00,  5.22s/it]


In [48]:
df_test_processed = df_test_processed.dropna()

In [49]:
df_test_processed['embs_np'] = df_test_processed['embs'].apply(lambda x: x.cpu().flatten().numpy())

In [50]:
df_test_processed = df_test_processed.reset_index()
df_test_new = pd.concat([df_test_processed['embs_np'], df_test_processed['Segment_num']], axis=1)
array_cols = pd.DataFrame(df_test_new['embs_np'].to_list(), columns=[f'array_feature_{i}' for i in range(len(df_test_new['embs_np'][0]))])
df_test_new = pd.concat([df_test_new.drop(columns=['embs_np']), array_cols], axis=1)

X_test = df_test_new.drop(columns=['Segment_num'])
y_test = df_test_new['Segment_num']

In [51]:
from sklearn.metrics import classification_report

y_pred = CatBoostModel_emb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.64      0.76       173
           1       0.35      0.50      0.41        12
           2       0.00      0.00      0.00         3
           3       0.30      0.60      0.40         5
           4       0.33      0.50      0.40        12
           5       0.40      0.89      0.55         9
           7       0.00      0.00      0.00         3
           8       0.25      0.50      0.33         4
           9       0.33      0.25      0.29         4
          10       0.50      0.62      0.56         8
          11       0.00      0.00      0.00         3
          12       0.15      0.67      0.25         3
          13       0.80      0.80      0.80         5
          14       0.33      0.62      0.43         8
          15       0.75      0.53      0.62        17
          16       0.83      0.81      0.82        31
          17       0.61      0.74      0.67        27
          18       0.25    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [52]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='macro')

0.41438178427412037

In [53]:
def predict_class(df_train, df_test):
    train_embs = np.vstack([emb.cpu().numpy() for emb in df_train['embs'].values])
    train_labels = df_train['Segment_num'].values
    
    def get_top_k_classes(test_emb, train_embs, train_labels, k=1):
        similarities = cosine_similarity(test_emb.reshape(1, -1), train_embs).flatten()
        top_k_indices = similarities.argsort()[-k:][::-1]
        top_k_classes = train_labels[top_k_indices]
        most_common_class = Counter(top_k_classes).most_common(1)[0][0]
        
        return most_common_class
    
    predicted_classes = df_test['embs'].apply(lambda emb: get_top_k_classes(emb.cpu().numpy(), train_embs, train_labels))
    
    return predicted_classes

In [58]:
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

In [59]:
preds = predict_class(df_train_processed, df_test_processed)

In [60]:
print(classification_report(df_test_processed['Segment_num'], preds))

              precision    recall  f1-score   support

           0       0.91      0.57      0.70       173
           1       0.18      0.33      0.24        12
           2       0.14      0.67      0.24         3
           3       0.25      0.60      0.35         5
           4       0.27      0.67      0.38        12
           5       0.53      1.00      0.69         9
           7       0.20      0.33      0.25         3
           8       0.20      0.25      0.22         4
           9       0.33      0.75      0.46         4
          10       0.40      0.50      0.44         8
          11       0.17      0.67      0.27         3
          12       0.14      0.33      0.20         3
          13       0.25      0.60      0.35         5
          14       0.47      0.88      0.61         8
          15       0.88      0.41      0.56        17
          16       0.92      0.77      0.84        31
          17       0.85      0.41      0.55        27
          18       0.22    

In [61]:
f1_score(df_test_processed['Segment_num'], preds, average='macro')

0.42182361938447205

In [64]:
df_train_processed.to_csv('embeddings.csv')